1. Data Cleaning

This notebook covers the preprocessing pipeline for the IPO event study.
It includes importing raw datasets, cleaning inconsistencies, calculating returns, and preparing final structured files for analysis.

1.1 Importing Raw Files 


(a) Load raw IPO sample files

(b) Load benchmark index data (NIFTY 50)

(c) Validate file formats and basic structure

In [9]:
import pandas as pd
import numpy as np
import os

# Set file paths
raw_path = "IPO Event Study Python Project/Data/Raw_Data/" 
processed_path =  "IPO Event Study Python Project/Data/Processed_Data/"


In [107]:
import pandas as pd
import numpy as np
import os

ipo_raw = pd.read_csv("../Data/Raw_Data/ipo_list_raw.csv")
nifty_raw = pd.read_csv("../Data/Raw_Data/NIFTY_50_raw.csv")

ipo_raw.head(), nifty_raw.head()




(             Company    Ticker Listing_Date  Issue_Price       Sector
 0  Tata Technologies  TATATECH   2023-11-30          500   Technology
 1     Mankind Pharma   MANKIND   2023-05-09         1080       Pharma
 2              Ixigo     IXIGO   2024-06-10           93  Travel-Tech
 3           Go Digit   GODIGIT   2024-05-23          278    Insurance
 4       Bikaji Foods    BIKAJI   2022-11-16          300         FMCG,
          Date  Close
 0  2023-12-20  21475
 1  2023-12-21  21502
 2  2023-12-22  21488
 3  2023-12-23  21515
 4  2023-12-24  21542)

In [6]:
ipo_raw.info()
ipo_raw.head()
ipo_raw.tail()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Company       5 non-null      object
 1   Ticker        5 non-null      object
 2   Listing_Date  5 non-null      object
 3   Issue_Price   5 non-null      int64 
 4   Sector        5 non-null      object
dtypes: int64(1), object(4)
memory usage: 332.0+ bytes


,Company,Ticker,Listing_Date,Issue_Price,Sector
0,Tata Technologies,TATATECH,2023-11-30,500,Technology
1,Mankind Pharma,MANKIND,2023-05-09,1080,Pharma
2,Ixigo,IXIGO,2024-06-10,93,Travel-Tech
3,Go Digit,GODIGIT,2024-05-23,278,Insurance
4,Bikaji Foods,BIKAJI,2022-11-16,300,FMCG


In [108]:
nifty_raw.info()
nifty_raw.head()
nifty_raw.tail()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 43 entries, 0 to 42
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Date    43 non-null     object
 1   Close   43 non-null     int64 
dtypes: int64(1), object(1)
memory usage: 820.0+ bytes


,Date,Close
38,2024-01-27,22305
39,2024-01-28,22330
40,2024-01-29,22355
41,2024-01-30,22385
42,2024-01-31,22410


1.2 Cleaning of IPO Samples  

(a) Standardize column names

(b) Remove duplicates

(c) Handle inconsistent date formats

(d) Validate listing dates

In [64]:
ipo_raw.columns = ipo_raw.columns.str.strip().str.lower()
ipo_raw['listing_date'] = pd.to_datetime(ipo_raw['listing_date'])
ipo_raw = ipo_raw.sort_values(by= 'listing_date')
ipo_raw = ipo_raw.drop_duplicates()
ipo_raw = ipo_raw.reset_index(drop=True)

ipo_raw.head()
ipo_raw.isna().sum()

company         0
ticker          0
listing_date    0
issue_price     0
sector          0
dtype: int64

1.3 Cleaning of NIFTY Data 

(a) Set correct datetime index

(b) Fix missing/incorrect price entries

(c) Format columns required for return calculation



In [109]:
nifty_raw.columns = ['date','close']
nifty_raw['date'] = pd.to_datetime(nifty_raw['date'])
nifty_raw = nifty_raw.sort_values(by='date').drop_duplicates().reset_index(drop=True)
print (nifty_raw.head())
nifty_raw.isna().sum()
nifty_raw[nifty_raw['date'].isna()]

nifty_raw = nifty_raw.dropna(subset=['date']).reset_index(drop=True)
nifty_raw.head()
nifty_raw.isna().sum()

        date  close
0 2023-12-20  21475
1 2023-12-21  21502
2 2023-12-22  21488
3 2023-12-23  21515
4 2023-12-24  21542


date     0
close    0
dtype: int64

1.4 Calculation of Daily Returns For NIFTY

 Formula : Rt = P1 - P0 / P0

(a) Calculate daily percentage returns

(b) Inspect return distribution

In [110]:
#Calculation of Daily Returns For NIFTY
nifty_raw['close'] = pd.to_numeric(nifty_raw['close'])
nifty_raw['mkt_return'] = nifty_raw['close'].pct_change()
nifty_raw.head()
 

,date,close,mkt_return
0,2023-12-20,21475,NaN
1,2023-12-21,21502,0.001257
2,2023-12-22,21488,-0.000651
3,2023-12-23,21515,0.001257
4,2023-12-24,21542,0.001255


1.5 IPO Sample Conversion, Formatting and Return Calculation

(a) Convert price columns to numeric

(b) Set trading dates as index

(c) Compute IPO stock daily returns

In [89]:
#Working with sample files now
#Sample1 Data loading
sample1 = pd.read_csv("../Data/Raw_Data/Sample_1.csv")
sample1.head()

#Conversion and Formatting
sample1['date'] = pd.to_datetime(sample1['date'])
sample1['close']= pd.to_numeric(sample1['close'])

#IPO Sample 1 return calculation
sample1['ipo_return'] = sample1['close'].pct_change()
sample1.head()
sample1.info()
sample1.shape
sample1

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   date        10 non-null     datetime64[ns]
 1   close       10 non-null     int64         
 2   ipo_return  9 non-null      float64       
dtypes: datetime64[ns](1), float64(1), int64(1)
memory usage: 372.0 bytes


,date,close,ipo_return
0,2024-01-01,102,NaN
1,2024-01-02,104,0.019608
2,2024-01-03,103,-0.009615
3,2024-01-04,105,0.019417
4,2024-01-05,108,0.028571
5,2024-01-08,110,0.018519
6,2024-01-09,113,0.027273
7,2024-01-10,111,-0.017699
8,2024-01-11,115,0.036036
9,2024-01-12,118,0.026087


In [85]:
#Sample2 Data loading
sample2 = pd.read_csv("../Data/Raw_Data/Sample_2.csv")
sample2.head()

#Conversion and Formatting
sample2['date'] = pd.to_datetime(sample2['date'])
sample2['close']= pd.to_numeric(sample2['close'])

#IPO Sample 2 return calculation
sample2['ipo_return'] = sample2['close'].pct_change()
sample2.head()
sample2.info()
sample2.shape
sample2 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   date        10 non-null     datetime64[ns]
 1   close       10 non-null     int64         
 2   ipo_return  9 non-null      float64       
dtypes: datetime64[ns](1), float64(1), int64(1)
memory usage: 372.0 bytes


,date,close,ipo_return
0,2024-01-01,220,NaN
1,2024-01-02,225,0.022727
2,2024-01-03,223,-0.008889
3,2024-01-04,228,0.022422
4,2024-01-05,232,0.017544
5,2024-01-08,231,-0.004310
6,2024-01-09,236,0.021645
7,2024-01-10,239,0.012712
8,2024-01-11,241,0.008368
9,2024-01-12,245,0.016598


In [111]:
#Data Merging (Created a new variable altogether for NIFTY to avoid any confusion with our orignally cleaned dataset)
nifty_returns = nifty_raw.copy()
nifty_returns['nifty_return'] = nifty_returns['close'].pct_change()
nifty_returns = nifty_returns.drop(columns=['mkt_return'])
nifty_returns

,date,close,nifty_return
0,2023-12-20,21475,NaN
1,2023-12-21,21502,0.001257
2,2023-12-22,21488,-0.000651
3,2023-12-23,21515,0.001257
4,2023-12-24,21542,0.001255
5,2023-12-25,21561,0.000882
6,2023-12-26,21590,0.001345
7,2023-12-27,21612,0.001019
8,2023-12-28,21640,0.001296
9,2023-12-29,21675,0.001617


1.6 Merging of IPO Datasets

(a) Align IPO datasets with benchmark data

(b) Merge on trading date

(c) Ensure event window alignment

In [112]:
#Sample1_merge
sample1_merged = sample1.merge(nifty_returns[['date','nifty_return']], on= 'date', how= 'left')
print (sample1_merged.head())
sample1_merged.isna().sum()

        date  close  ipo_return  nifty_return
0 2024-01-01    102         NaN      0.001384
1 2024-01-02    104    0.019608      0.000691
2 2024-01-03    103   -0.009615     -0.000460
3 2024-01-04    105    0.019417      0.001520
4 2024-01-05    108    0.028571      0.001104


date            0
close           0
ipo_return      1
nifty_return    0
dtype: int64

In [113]:
#Sample2_merge
sample2_merged = sample2.merge(nifty_returns[['date','nifty_return']], on='date',how='left')
print (sample2_merged.head())
sample2_merged.isna().sum()

        date  close  ipo_return  nifty_return
0 2024-01-01    220         NaN      0.001384
1 2024-01-02    225    0.022727      0.000691
2 2024-01-03    223   -0.008889     -0.000460
3 2024-01-04    228    0.022422      0.001520
4 2024-01-05    232    0.017544      0.001104


date            0
close           0
ipo_return      1
nifty_return    0
dtype: int64

1.7 Cleaning Missing Values

(a) Forward-fill where market holidays interrupt data

(b) Drop rows where alignment is not possible

(c) Validate final merged dataset

In [114]:
#Fixing Nifty NaNs using Forward Fill
#Why not backward fill? = We never backfill index returns. Because this corrupts pre-event info. But forward filling after the lisiting date is valid because markets don't regress.

sample1_merged['nifty_return'] = sample1_merged['nifty_return'].fillna(method='ffill')
sample2_merged['nifty_return'] = sample2_merged['nifty_return'].fillna(method='ffill')
print (sample1_merged)
print (sample2_merged)

        date  close  ipo_return  nifty_return
0 2024-01-01    102         NaN      0.001384
1 2024-01-02    104    0.019608      0.000691
2 2024-01-03    103   -0.009615     -0.000460
3 2024-01-04    105    0.019417      0.001520
4 2024-01-05    108    0.028571      0.001104
5 2024-01-08    110    0.018519      0.001696
6 2024-01-09    113    0.027273      0.001235
7 2024-01-10    111   -0.017699     -0.001005
8 2024-01-11    115    0.036036      0.001464
9 2024-01-12    118    0.026087      0.001279
        date  close  ipo_return  nifty_return
0 2024-01-01    220         NaN      0.001384
1 2024-01-02    225    0.022727      0.000691
2 2024-01-03    223   -0.008889     -0.000460
3 2024-01-04    228    0.022422      0.001520
4 2024-01-05    232    0.017544      0.001104
5 2024-01-08    231   -0.004310      0.001696
6 2024-01-09    236    0.021645      0.001235
7 2024-01-10    239    0.012712     -0.001005
8 2024-01-11    241    0.008368      0.001464
9 2024-01-12    245    0.016598   

C:\Users\SAINI JI\AppData\Local\Temp\ipykernel_13048\814731324.py:4: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  sample1_merged['nifty_return'] = sample1_merged['nifty_return'].fillna(method='ffill')
C:\Users\SAINI JI\AppData\Local\Temp\ipykernel_13048\814731324.py:5: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  sample2_merged['nifty_return'] = sample2_merged['nifty_return'].fillna(method='ffill')


1.8 Cleaned Files Ready For Export

(a) Export cleaned IPO return files

(b) Export cleaned NIFTY returns

(c) Verify export consistency

In [116]:
#Making your CLEANED files ready for exporting
#Export Sample1
sample1_merged.to_csv("clean_sample1.csv", index=False)

#Export Sample2
sample2_merged.to_csv("clean_sample2.csv", index=False)

#Export cleaned NIFTY dataset
nifty_returns.to_csv("clean_nifty_returns.csv", index=False)
